#### Install dependencies

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import shutil

# Define the paths for aiohttp files
aiohttp_path = '/opt/conda/lib/python3.10/site-packages/aiohttp*'
aiohttp_dist_info_path = '/opt/conda/lib/python3.10/site-packages/aiohttp-3.9.1.dist-info'

# Remove aiohttp package directories and files
for path in [aiohttp_path, aiohttp_dist_info_path]:
    shutil.rmtree(path, ignore_errors=True)



In [21]:
#!pip install -r -q requirements.txt

!pip -q install torch==2.2.0
!pip -q install typer==0.9.1

!pip -q install pyannote.audio # diarization
!pip -q install pydub # segmentation
!pip -q install transformers # transcription

!pip install modelbit

In [4]:
#from transcription import Recording, Transcription

from huggingface_hub import HfApi
available_pipelines = [p.modelId for p in HfApi().list_models(filter="pyannote-audio-pipeline")]
list(filter(lambda p: p.startswith("pyannote/"), available_pipelines))

from pyannote.audio import Pipeline
pipeline_diar = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token=True)

import torch
import torchaudio
import base64
from io import BytesIO

from pyannote.core import Annotation

from pydub import AudioSegment

from transformers import pipeline

transcriber = pipeline(model="openai/whisper-large-v2", return_timestamps=False)

config.yaml:   0%|          | 0.00/469 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/399 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/26.6M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/221 [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

#### Log into the modelbit workspace

In [6]:
import modelbit
mb = modelbit.login()

#### Training job

Dataset

In [7]:
AUDIO_FILE = f"/kaggle/input/movie-audio/audio.mp3"

pipeline_diar.to(torch.device("cuda"))


In [10]:
# Read the MP3 file in binary mode
with open(AUDIO_FILE, "rb") as file:
    mp3_data = file.read()

# Base64 encoded string
encoded_data = base64.b64encode(mp3_data).decode('utf-8')

print(encoded_data[:100])

SUQzBAAAAAABAFRYWFgAAAASAAADbWFqb3JfYnJhbmQAZGFzaABUWFhYAAAAEQAAA21pbm9yX3ZlcnNpb24AMABUWFhYAAAAHAAA


The Pipeline

In [63]:
# # Decode the base64 encoded string
# decoded_data = base64.b64decode(encoded_data)

# # Write the decoded audio data to a temporary file buffer
# # Create a BytesIO object from the byte array
# with BytesIO(decoded_data) as audio_buffer:
#     audio_buffer.seek(0)
#     # Load audio with pydub for easy slicing
#     audio = AudioSegment.from_file(audio_buffer)
#     audio.export("out.mp3", format="mp3")

#     # Run diarization pipeline
#     dia = pipeline_diar("out.mp3")
#     assert isinstance(dia, Annotation)

#     # Prepare a list to store data
#     data = []
    
#     for i, (speech_turn, track, speaker) in enumerate(dia.itertracks(yield_label=True)):
#         print(f"{speech_turn.start:4.1f} {speech_turn.end:4.1f} {speaker}")

#         # Extract start and end times
#         start_time, end_time = speech_turn.start, speech_turn.end

#         # Extract the segment of the audio
#         segment = audio[int(start_time*1000):int(end_time*1000)].export("out.mp3", format="mp3")
#         text = transcriber("out.mp3")['text']

#         # Append the data to the list
#         data.append({
#             "Start Time": start_time,
#             "End Time": end_time,
#             "Speaker": speaker,
#             "Transcription": text
#             })
#         print(data)

In [11]:
def transcribe_recording(base64_encoded_audio):
    """
    Transcribes audio data given as base64 encoded byte string into text and yields
    the transcribed text incrementally as it becomes available.

    Parameters:
    - base64_audio (str): Base64 encoded byte string of audio data.

    Yields:
    - str: Transcribed text incrementally as it becomes available.
    """
    # Decode the base64 encoded string
    base64_decoded_audio = base64.b64decode(base64_encoded_audio)
    
    with BytesIO(base64_decoded_audio) as audio_buffer:
        audio_buffer.seek(0)
        # Load audio with pydub for easy slicing
        audio = AudioSegment.from_file(audio_buffer)
        audio.export("out.mp3", format="mp3")

        # Run diarization pipeline
        dia = pipeline_diar("out.mp3")
        assert isinstance(dia, Annotation)
        # Prepare a list to store data
        data = []

        for i, (speech_turn, track, speaker) in enumerate(dia.itertracks(yield_label=True)):
            # Extract start and end times
            start_time, end_time = speech_turn.start, speech_turn.end

            # Extract and transcribe the audio segment
            segment_audio = audio[int(start_time * 1000):int(end_time * 1000)]
            segment_audio.export("out.mp3", format="mp3")
            text = transcriber("out.mp3")['text']

            # Append the data to the list
            data.append({
                "Start Time": start_time,
                "End Time": end_time,
                "Speaker": speaker,
                "Transcription": text
            })

    return data


In [12]:
transcribe_recording(encoded_data)

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


[{'Start Time': 4.31721875,
  'End Time': 8.04659375,
  'Speaker': 'SPEAKER_00',
  'Transcription': " So last night I was watching this movie. Have you ever seen the movie North and South? I haven't."},
 {'Start Time': 7.64159375,
  'End Time': 8.02971875,
  'Speaker': 'SPEAKER_01',
  'Transcription': ' I have it.'},
 {'Start Time': 8.04659375,
  'End Time': 8.19846875,
  'Speaker': 'SPEAKER_01',
  'Transcription': ' you'},
 {'Start Time': 8.384093750000002,
  'End Time': 11.489093750000002,
  'Speaker': 'SPEAKER_00',
  'Transcription': " It's kind of like Pride and Prejudice. It's really good. I..."},
 {'Start Time': 11.539718750000002,
  'End Time': 14.42534375,
  'Speaker': 'SPEAKER_01',
  'Transcription': " I haven't even seen Pride and Prejudice. I'm like way out of the loop on them."},
 {'Start Time': 12.872843750000001,
  'End Time': 13.36221875,
  'Speaker': 'SPEAKER_00',
  'Transcription': ' Alors...'},
 {'Start Time': 14.492843750000002,
  'End Time': 15.707843750000002,
  'S

#### Deploy the pipeline

In [14]:
# deploy transcription
mb.deploy(transcribe_recording, 
          require_gpu=True, 
#           system_packages=["ffmpeg"],
         python_packages=["tensorflow==2.16.1"],
#                            "pyannote.audio==3.2.0",
#                            "pydub==0.25.1",
#                            "transformers==4.40.2"]
         )

In [20]:
modelbit.get_inference(
  region="ap-south-1",
  workspace="juristone",
  deployment="transcribe_recording",
  data=encoded_data
)